For individuals or pairs

Please write an iPython notebook that represents words and articles in the Reuters corpus as vectors and clusters the article vectors. Please use the nltk.corpus.reuters "training" documents (as shown in reuters.fileids()) in one (and only one) of the categories 

`ship, trade, interest, money-fx, crude`

In [1]:
import nltk
import os
#nltk.download('reuters')
from nltk.corpus import reuters
from collections import Counter
import pandas as pd
import numpy as np

In [ ]:
#print(reuters.fileids())
#money = reuters.fileids("money-fx")
#dir(reuters)
# # separate train from test
# positive_train = [f for f in money_files if "training/" in f]
# positive_test = [f for f in money_files if "test/" in f]

# all_files = reuters.fileids()
# all_train = [f for f in all_files if "training/" in f]
# all_test = [f for f in all_files if "test/" in f]

# # take difference of sets to get negative classes
# negative_train = list(set(all_train).difference(set(positive_train)))
# negative_test = list(set(all_test).difference(set(positive_test)))

1. Create a term-document matrix containing a row for every word in the corpus vocabulary and a column for each document, where each entry is the tf-idf score of a word for a document.
    
    

In [5]:
superset_words = set()
corpus_wordcounts = Counter()
wordcounts_per_article = {}

# subset the dataset
article_subset = [article for article in reuters.fileids() if np.random.uniform() < 0.10]
print("size of subset is ", len(article_subset))

for article in article_subset:
    words_in_article = reuters.words(article)
    # get rid of symbols and numbers, lowercase words
    words_in_article = [word.lower() for word in words_in_article if word.isalnum() and not word.isnumeric() 
                        and not word.startswith("0")  # omg this is a terrible hack :-O
                        and not word.startswith("1")
                        and not word.startswith("4")
                        and not word.startswith("5")
                        and not word.startswith("8")]
    # construct superset of all words in all articles
    superset_words = superset_words | set(words_in_article)
    
    # words counts per article
    article_word_counts = Counter(words_in_article)
    wordcounts_per_article[article] = article_word_counts
    # word counts in whole corpus
    corpus_wordcounts += article_word_counts

# termfreqs_per_article = {}

# for article, counts in wordcounts_per_article.items():
#     total_words_in_aritcle = sum(counts.values())
#     article_termfreqs = Counter()
#     for word in superset_words:
#         word_count = counts[word]
#         # term freqs per article are log-scaled
#         article_termfreqs[word] = np.log(word_count+1)
#     termfreqs_per_article[article] = article_termfreqs


# N_documents_in_corpus = len(wordcounts_per_article)  # count num articles
# # docfreq[t] is the number of documents in which term t occurs
# docfreqs = Counter()
# for article, counts in wordcounts_per_article.items():
#     for word in superset_words:
#         if counts[word] > 0:
#             docfreqs[word] += 1
# inverse_docfreqs = Counter()
# for word, docfreq in docfreqs.items():
#     inverse_docfreq = np.log(N_documents_in_corpus/docfreq)
#     inverse_docfreqs[word] = inverse_docfreq


# tf_idf_per_article = 

# article_termfreqs
    
# TODO: term freq is log_10(1+count(t,d)) if count > 0, else 0
# TODO: tf-idf = tf * idf 

size of subset is  1042


In [6]:
superset_words = sorted(list(superset_words))  # get words in alphabetical order

In [7]:
# raw_term_freqs = {}
# for article in money:
#     term_frequencies = [wordcounts_per_article[article][word] for word in superset_words]
#     raw_term_freqs[article] = term_frequencies
    
term_freqs = {}
for article, raw_counts in wordcounts_per_article.items():
    scaled_counts = Counter()
    for word, raw_count in raw_counts.items():
        if raw_count > 0:
            count = 1 + np.log10(raw_count)
        else:
            count = 0
        scaled_counts[word] = count
    """
    Counters are unordered, so do list comprehension to get the vals in sorted order
    """
    term_freqs[article] = [scaled_counts[word] for word in superset_words]
    
term_freq_df = pd.DataFrame.from_dict(term_freqs)
term_freq_df["word"] = superset_words
term_freq_df.set_index("word", inplace=True)

In [9]:
def count_N_docs_term_present(row):
    times_term_present = 0
    for count in row:
        if count > 0:
            times_term_present += 1
            
    return times_term_present

N_docs_term_present = term_freq_df.apply(count_N_docs_term_present, axis=1)
N_documents = len(article_subset)
inverse_document_frequencies = np.log10(N_documents / N_docs_term_present)
tf_idf_df = term_freq_df.apply(lambda column: column * inverse_document_frequencies, axis=0)

In [10]:
print(tf_idf_df.shape)
tf_idf_df

(9443, 1042)


,test/14828,test/14861,test/14865,test/14888,test/14890,test/14959,test/14971,test/14993,test/15052,test/15092,...,training/9822,training/984,training/9878,training/9880,training/9939,training/9952,training/9977,training/9985,training/9988,training/999
word,,,,,,,,,,,,,,,,,,,,,
21p,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,...,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0
25p,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,...,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0
25th,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,...,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0
27p,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,...,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0
2nd,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,...,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0
2p,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,...,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0
32nd,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,...,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0
33p,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,...,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0
37p,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,...,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0


2. Reduce the size of the matrix. Compute the maximum tf-idf score for each word and keep the 500 rows with the top 500 maxima. Did that remove the maximum tf-idf score of any column? Comment. 
    

In [11]:
max_score_per_word = tf_idf_df.apply(max, axis=1)
top_500_maxima = max_score_per_word.sort_values(ascending=False).head(500)
top_500_maxima_words = top_500_maxima.index

labels_to_keep = list(top_500_maxima_words)

reduced_tf_idf_df = tf_idf_df.loc[labels_to_keep]

print(reduced_tf_idf_df.shape)
reduced_tf_idf_df

(500, 1042)


,test/14828,test/14861,test/14865,test/14888,test/14890,test/14959,test/14971,test/14993,test/15052,test/15092,...,training/9822,training/984,training/9878,training/9880,training/9939,training/9952,training/9977,training/9985,training/9988,training/999
word,,,,,,,,,,,,,,,,,,,,,
texaco,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
resorts,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
lpg,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
audi,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
mcivor,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
sprint,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
bally,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
sosnoff,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
bil,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [12]:
# TODO figure out how to do this

max_score_per_document = tf_idf_df.apply(max, axis=0)
max_score_per_document

test/14828       3.926336
test/14861       3.143038
test/14865       4.834805
test/14888       5.127267
test/14890       5.127267
test/14959       3.926336
test/14971       3.017868
test/14993       5.127267
test/15052       4.457757
test/15092       2.751389
test/15093       3.534687
test/15097       4.070428
test/15128       3.926336
test/15185       3.926336
test/15200       3.805211
test/15206       5.568262
test/15219       3.715013
test/15221       3.017868
test/15246       5.743274
test/15274       4.070428
test/15296       4.457757
test/15344       4.013099
test/15383       3.926336
test/15388       4.295672
test/15389       5.012832
test/15400       3.592944
test/15404       4.834805
test/15436       3.588160
test/15448       3.308333
test/15460       4.965235
                   ...   
training/9408    6.035735
training/9417    3.926336
training/9425    3.017868
training/9432    4.457757
training/9511    2.826839
training/9532    4.070428
training/9547    3.017868
training/955

3. Cluster the document vectors into five clusters using an unsupervised algorithm like k-means. Create a 5x5 matrix that compares each cluster to the each of the above five categories, using the Jaccard Index (see below). Comment.

The Jaccard Index compares two sets A and B using the formula

J(A,B) = |A and B | / |A or B|

In [13]:
from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=5).fit(reduced_tf_idf_df)
# todo plot clusters

In [14]:
#superset_words

['21p',
 '25p',
 '25th',
 '27p',
 '2nd',
 '2p',
 '32nd',
 '33p',
 '37p',
 '38p',
 '3p',
 '3rd',
 '6p',
 '75p',
 '9p',
 'a',
 'aa',
 'aart',
 'ab',
 'abandon',
 'abandoning',
 'abandons',
 'abastecimiento',
 'abate',
 'abated',
 'abdel',
 'abdul',
 'abdullah',
 'abide',
 'abiding',
 'ability',
 'able',
 'abn',
 'abnn',
 'abnormal',
 'abolish',
 'about',
 'above',
 'abroad',
 'abrupt',
 'absence',
 'absolute',
 'absorb',
 'absorbed',
 'abuses',
 'abx',
 'academic',
 'accelerate',
 'accelerated',
 'acceleration',
 'accentuating',
 'accept',
 'acceptable',
 'acceptance',
 'acceptances',
 'accepted',
 'accepting',
 'accepts',
 'access',
 'accession',
 'accident',
 'accommodation',
 'accommodative',
 'accompanied',
 'accomplish',
 'accomplished',
 'accomplishments',
 'accord',
 'accordance',
 'according',
 'accordingly',
 'accords',
 'account',
 'accountants',
 'accounted',
 'accounting',
 'accounts',
 'accredited',
 'accrued',
 'accruing',
 'accumulated',
 'accumulating',
 'accurate',
 'acc

4. Try clustering the words and comparing those clusters to the categories, too. Comment on the results.



In [ ]:
# take the transpose of the matrix and  